In [34]:
from PIL import Image
import pandas as pd
import os
import numpy as np
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from skimage.transform import rotate
from skimage.feature import canny
from scipy.stats import zscore
from skimage.measure import regionprops, label
!pip install opencv-python
import cv2
from scipy.optimize import curve_fit
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import glob
import os
from scipy.stats import linregress

In [42]:
def load_csv(file_path):
    # Read the CSV file into a df
    df = pd.read_csv(file_path, header=0, index_col=0)
    return df

def center_of_intensity(coordinates, intensities):
    numerator = np.sum(coordinates * intensities)
    denominator = np.sum(intensities)
    if denominator == 0:
        return np.nan
    return numerator / denominator



def calculate_coi(file_path, dividing_line, height):
    data = load_csv(file_path)
    z_cord = data.index.values
    x_cord = data.columns.values.astype(float)
    intensity_values = data.values

    if dividing_line is not None:
        focal_mask = z_cord >= dividing_line
        surface_mask = (z_cord < dividing_line) & (z_cord >= -height * 1000)
    else:
        focal_mask = np.ones_like(z_cord, dtype=bool)
        surface_mask = np.zeros_like(z_cord, dtype=bool)
    
    focal_intensities = intensity_values[focal_mask]

    # Apply cutoff and subtract linear background for focal intensities
    focal_sum = np.sum(focal_intensities, axis=1)
    focal_max = np.max(focal_sum)
    cutoff = (1 / 100) * focal_max
    f_indices = np.where(focal_sum >= cutoff)[0]
    f_z_coords = z_cord[focal_mask][f_indices]
    f_intensities = focal_intensities[f_indices]
    f_intensity_sum = focal_sum[f_indices]
    f_intensity_sum, _, _, _ = subtract_linear_background(f_z_coords, f_intensity_sum)

    coi_x_focal = center_of_intensity(x_cord, np.sum(f_intensities, axis=0))
    coi_z_focal = center_of_intensity(f_z_coords, f_intensity_sum)
    
    if dividing_line is not None:
        surface_intensities = intensity_values[surface_mask]
        surface_sum = np.sum(surface_intensities, axis=1)
        s_max = np.max(surface_sum)
        s_cutoff = (1 / 100) * s_max
        s_indices = np.where(surface_sum >= s_cutoff)[0]
        s_z_cords = z_cord[surface_mask][s_indices]
        s_intensity = surface_intensities[s_indices]
        s_intensity_sum = surface_sum[s_indices]
        s_intensity_sum, _, _, _ = subtract_linear_background(s_z_cords, s_intensity_sum)

        coi_x_surface = center_of_intensity(x_cord, np.sum(s_intensity, axis=0))
        coi_z_surface = center_of_intensity(s_z_cords, s_intensity_sum)
        
        return (coi_x_focal, coi_z_focal), (coi_x_surface, coi_z_surface)
    else:
        return (coi_x_focal, coi_z_focal), (None, None)



def find_dividing_line(z_coords, intensity_profile, lower_boundary, upper_boundary):
    # Apply the boundaries
    within_bounds = (z_coords >= lower_boundary) & (z_coords <= upper_boundary)
    bounded_z_coords = z_coords[within_bounds]
    bounded_profile = intensity_profile[within_bounds]
    
    # Find the absolute minimum within the boundaries
    min_index = np.argmin(bounded_profile)
    dividing_line = bounded_z_coords[min_index]
    return dividing_line

def subtract_linear_background(z_coords, intensity_profile):
    # Fit a linear regression to the bottom 5% of the intensity profile
    num_points = len(intensity_profile)
    num_fit_points = max(1, num_points // 20)  # Bottom 5%
    sorted_indices = np.argsort(intensity_profile)
    fit_indices = sorted_indices[:num_fit_points]
    
    unique_fit_points = np.unique(z_coords[fit_indices])
    if len(unique_fit_points) < 2:
        # Not enough unique points for a valid linear regression
        return intensity_profile, 0, 0, np.zeros_like(z_coords)
    
    slope, intercept, _, _, _ = linregress(z_coords[fit_indices], intensity_profile[fit_indices])
    linear_background = slope * z_coords + intercept
    
    return intensity_profile - linear_background, slope, intercept, linear_background


In [51]:


def spatial_z(directory, height, output_dir, cutoff_percentage=1):
    # Dictionary to store data for each delay time
    data_dict = {}

    for filename in sorted(os.listdir(directory)):
        if filename.endswith('.csv'):
            # Extract delay time and trial number from the filename
            delay_time = int(filename.split('_')[0].replace('ns', ''))
            trial = int(filename.split('_')[1].split('.')[0])
            file_path = os.path.join(directory, filename)
            
            # Load the CSV file into a DataFrame
            df = load_csv(file_path)
            
            # Separate coordinates and intensity data
            z_coords = df.index.astype(float)
            intensity_data = df.values
            
            # Sum along the y direction (axis=1)
            spatial_profile = np.sum(intensity_data, axis=1)
            
            # Store the spatial profile and z_coords in the dictionary
            if delay_time not in data_dict:
                data_dict[delay_time] = {'profiles': [], 'z_coords': z_coords}
            data_dict[delay_time]['profiles'].append(spatial_profile)
    
    # Process each delay time to calculate the average and standard deviation
    dividing_lines = {}
    
    lower_boundary = -height * 1000  # Convert mm to µm
    upper_boundary = 0
    
    for delay_time, data in data_dict.items():
        all_profiles = np.array(data['profiles'])
        avg_profile = np.mean(all_profiles, axis=0)
        std_profile = np.std(all_profiles, axis=0)
        z_coords = data['z_coords']

        # Apply the cutoff
        max_intensity = np.max(avg_profile)
        cutoff_value = (cutoff_percentage / 100) * max_intensity
        filtered_indices = np.where(avg_profile >= cutoff_value)[0]

        if len(filtered_indices) == 0:
            dividing_line = 0
        else:
            # Apply cutoff to data
            filtered_z_coords = z_coords[filtered_indices]
            filtered_profile = avg_profile[filtered_indices]

            # Subtract the linear background
            filtered_profile, slope, intercept, linear_background = subtract_linear_background(filtered_z_coords, filtered_profile)
            
            # Find the dividing line within the filtered data
            dividing_line = find_dividing_line(filtered_z_coords, filtered_profile, lower_boundary, upper_boundary)
        
        dividing_lines[delay_time] = dividing_line
        
        # Optional: Plot the filtered data for verification
        plt.figure()
        plt.errorbar(filtered_z_coords, filtered_profile, yerr=std_profile[filtered_indices], fmt='-', capsize=5)
        plt.axvline(x=dividing_line, color='red', linestyle='--', label=f'Dividing Line at z = {dividing_line} μm')
        plt.plot(filtered_z_coords, linear_background, 'g--', label='Linear Background')
        plt.xlabel('Z position (μm)')
        plt.ylabel('Intensity')
        # plt.yscale('log')
        plt.title(f'Spatial Profile along Z at {height}mm and {delay_time}ns')
        plt.legend()
        
        output_path = os.path.join(output_dir, f'Spatial_along_z_at_{height}mm/{delay_time}ns.png')
        plt.savefig(output_path)
        plt.close()

    return dividing_lines

In [37]:
base_directory = '/Users/YK/Desktop/24_06_28_water_air_late_cam2/'
output_dir = os.path.join(base_directory, '3mm', 'trash')
dividing_lines = spatial_z(os.path.join(base_directory, '3mm', 'CSV'), 3, output_dir)

In [52]:

def plot_coi_vs_time(coi_data, heights, output_dir, direction, plot_type):
    plt.figure(figsize=(12, 8))
    
    for height in heights:
        delays = []
        avg_cois = []
        std_cois = []
        for delay_time, cois in sorted(coi_data[height]):
            delays.append(delay_time)
            avg_cois.append(np.mean(cois))
            std_cois.append(np.std(cois))
        
        plt.errorbar(delays, avg_cois, yerr=std_cois, fmt='o-', capsize=5, label=f'Height {height}mm')
    
    plt.xlabel('Delay Time (ns)', fontsize=14)
    plt.xscale('log')
    plt.ylabel(f'Center of Intensity ({direction})', fontsize=14)
    plt.title(f'Center of Intensity in {direction}-Direction over Delay Time ({plot_type})', fontsize=16)
    plt.legend()
    plt.grid(True, linestyle='--', linewidth=0.5)
    
    # Save the plot
    output_path = os.path.join(output_dir, f'COI_{direction}_vs_time_{plot_type}.png')
    plt.savefig(output_path)
    plt.close()
    
def spatial_all_coi(base_directory, media, output_dir):
    heights = list(range(3, 6, 1)) + ['Air']
    coi_x_data_focal = {height: [] for height in heights}
    coi_z_data_focal = {height: [] for height in heights}
    coi_x_data_surface = {height: [] for height in heights}
    coi_z_data_surface = {height: [] for height in heights}
    
    for height in heights:
        data_directory = os.path.join(base_directory, f'{height}mm', 'CSV') if height != 'Air' else os.path.join(base_directory, 'Air', 'CSV')
        if not os.path.exists(data_directory):
            print(f'Directory {data_directory} does not exist, skipping.')
            continue
        
        if height != 'Air':
            dividing_lines = spatial_z(data_directory, height, output_dir)
        
        print(f'Processing {media} at {height}mm...')
        
        for filename in sorted(os.listdir(data_directory)):
            if filename.endswith('.csv'):
                delay_time = int(filename.split('_')[0].replace('ns', ''))
                file_path = os.path.join(data_directory, filename)
                
                if height != 'Air':
                    dividing_line = dividing_lines[delay_time]
                    (coi_x_focal, coi_z_focal), (coi_x_surface, coi_z_surface) = calculate_coi(file_path, dividing_line, height)
                else:
                    (coi_x_focal, coi_z_focal), _ = calculate_coi(file_path, None, height)
                
                if not any(item[0] == delay_time for item in coi_x_data_focal[height]):
                    coi_x_data_focal[height].append((delay_time, []))
                    coi_z_data_focal[height].append((delay_time, []))
                    if height != 'Air':
                        coi_x_data_surface[height].append((delay_time, []))
                        coi_z_data_surface[height].append((delay_time, []))
                
                for item in coi_x_data_focal[height]:
                    if item[0] == delay_time:
                        item[1].append(coi_x_focal)
                        break
                for item in coi_z_data_focal[height]:
                    if item[0] == delay_time:
                        item[1].append(coi_z_focal)
                        break
                if height != 'Air':
                    for item in coi_x_data_surface[height]:
                        if item[0] == delay_time:
                            item[1].append(coi_x_surface)
                            break
                    for item in coi_z_data_surface[height]:
                        if item[0] == delay_time:
                            item[1].append(coi_z_surface)
                            break

    for height in heights:
        coi_x_data_focal[height].sort()
        coi_z_data_focal[height].sort()
        if height != 'Air':
            coi_x_data_surface[height].sort()
            coi_z_data_surface[height].sort()
    
    # Plot COI vs. time for both focal and surface plasma separately
    plot_coi_vs_time(coi_x_data_focal, heights, output_dir, 'X', 'Focal Plane Plasma')
    plot_coi_vs_time(coi_z_data_focal, heights, output_dir, 'Z', 'Focal Plane Plasma')
    plot_coi_vs_time(coi_x_data_surface, [h for h in heights if h != 'Air'], output_dir, 'X', 'Surface Plasma')
    plot_coi_vs_time(coi_z_data_surface, [h for h in heights if h != 'Air'], output_dir, 'Z', 'Surface Plasma')


In [53]:
directory = '/Users/YK/Desktop/24_06_28_water_air_late_cam2'
output_dir2 = '/Users/YK/Desktop/24_06_28_water_air_late_cam2/COI'

In [54]:
spatial_all_coi(directory, 'media', output_dir2)

Processing media at 3mm...
Processing media at 4mm...
Processing media at 5mm...
Processing media at Airmm...
